# Implementing a CNN for Human Activity Recognition in Keras and Tensorflow
This notebook was based of tutorial on https://www.kdnuggets.com/2016/11/implementing-cnn-human-activity-recognition-tensorflow.html

The notebook shows how CNN was implemented using Keras and Tensorflow. The notebook uses Actitracker data set released by Wireless Sensor Data Mining (WISDM) lab. This dataset contains six daily activities collected in a controlled laboratory environment. The activities include jogging, walking, ascending stairs, descending stairs, sitting and standing. The data is collected from 36 users using a smartphone in their pocket with the 20Hz sampling rate.

## Libraries and Helper functions.
Here are the libraries that were imported for this notebook. These library will help with reading and normalising the data.

In [2]:
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
import keras

Using TensorFlow backend.


The read_data function will return the data as a Pandas data frame. We also need to normalise each of the accelerometer component (i.e. x, y and z) using feature_normalize method.

In [3]:
def read_data(file_path):
    column_names = ['user-id','activity','timestamp', 'x-axis', 'y-axis', 'z-axis']
    data = pd.read_csv(file_path,header = None, names = column_names)
    return data

def feature_normalize(dataset):
    mu = np.mean(dataset,axis = 0)
    sigma = np.std(dataset,axis = 0)
    return (dataset - mu)/sigma

Now using those functions the dataset can be read and normalized.

In [4]:
dataset = read_data('actitracker_raw.txt')
dataset['x-axis'] = feature_normalize(dataset['x-axis'])
dataset['y-axis'] = feature_normalize(dataset['y-axis'])
dataset['z-axis'] = feature_normalize(dataset['z-axis'])

## Preparing the data
For CNN the data has to be split into batches. We create functions below to create fixed sized segments from the raw signals. The windows function will generate indexes as specified by the size parameter by moving over the signal by fixed step size. The window size used is 90, which equals to 4.5 seconds of data and as we are moving each time by 45 points the step size is equal to 2.25 seconds. The label (activity) for each segment will be selected by the most frequent class label presented in that window. The segment_signal will generate fixed size segments and append each signal component along the third dimension so that the input dimension will be [ number of segments, input width and input channel].

In [5]:
def windows(data, size):
    start = 0
    while start < data.count():
        yield start, start + size
        start += (size / 2)
        
def segment_signal(data,window_size = 90):
    segments = np.empty((0,window_size,3))
    labels = np.empty((0))
    for (start, end) in windows(data["timestamp"], window_size):
        x = data["x-axis"][start:end]
        y = data["y-axis"][start:end]
        z = data["z-axis"][start:end]
        if(len(dataset["timestamp"][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([x,y,z])])
            labels = np.append(labels,stats.mode(data["activity"][start:end])[0][0])
    return segments, labels

segments, labels = segment_signal(dataset)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

/usr/local/lib/python2.7/dist-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


### Train test split
Now we are able to create fixed sized segments and do train test split, to test out the accuracy of our network. We will be using 70% of our data to train and 30% to use for accurarcy test.

In [6]:
train_test_split = np.random.rand(len(segments)) < 0.70
train_x = segments[train_test_split]
train_y = labels[train_test_split]
test_x = segments[~train_test_split]
test_y = labels[~train_test_split]

## Implementing CNN
The image below describes how the CNN would look like. This image was taken from the tutorial source that was mentioned above. The model will consist of one convolution layer followed by max pooling and another convolution layer. After that, the model will have fully connected layer which is connected to Softmax layer. 

![alt text](https://www.kdnuggets.com/wp-content/uploads/har-cnn.png)

### Creating the model
The model can be seen implemented below using Keras.

In [7]:
#Input details
input_width = 90
num_labels = 6
num_channels = 3

#Layer details
batch_size = 10
kernel_size = 60
depth = 60
num_hidden = 1000

#Training details
learning_rate = 0.0001
training_epochs = 8

#Model layers
inp = keras.layers.Input(shape=(input_width,num_channels))
Conv1 = keras.layers.Conv1D(depth,kernel_size,padding="valid",activation="relu",kernel_initializer="TruncatedNormal", use_bias=True, bias_initializer="Zeros")(inp)
MaxPool1= keras.layers.MaxPool1D(pool_size= 20, strides=2)(Conv1)
Conv2= keras.layers.Conv1D(depth//10,6,padding="valid", activation="relu",kernel_initializer="TruncatedNormal",use_bias=True, bias_initializer="Zeros")(MaxPool1)
Flat = keras.layers.Flatten()(Conv2)
hidden = keras.layers.Dense(num_hidden,activation='tanh')(Flat)
out = keras.layers.Dense(num_labels,activation='softmax')(hidden)

#Defining the model
model = keras.Model(inputs=inp, outputs=out)

The model will be compiled using Adam optimizer, and cross-entropy loss function.

In [8]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

Below as can be seen the model is being trained and then tested for accuracy.

In [9]:
model.fit(train_x, train_y,                # Train the model using the training set...
          batch_size=batch_size, epochs=training_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation
model.evaluate(test_x, test_y, verbose=2)  # Evaluate the trained model on the test set!

Train on 4824 samples, validate on 537 samples
Epoch 1/8
4824/4824 [==============================] - 2s 420us/step - loss: 0.4797 - acc: 0.8114 - val_loss: 0.7046 - val_acc: 0.7542

[0.22090678580588424, 0.93623438173201201]

# Visualizing The Performance of The CNN
This part will visualize the performance of the CNN above, this can be done visualisng the confussion matrix. A confusion matrix is a summary of prediction results on a classification problem, where the number of correct and incorrect predictions are summarized with count values and broken down by each class. We would do this by importing sciket learn library and use its confusion matrix function with using our predicted values.

In [21]:
from sklearn.metrics import confusion_matrix


pred_Y = model.predict(train_x)
pred_y = np.argmax(pred_Y, axis=1)
print(confusion_matrix(np.argmax(train_y,axis=1), pred_y))

[[ 411   16    7    0   30   13]
 [   0 1635    0    0    2    0]
 [   1    0  201    0    0    0]
 [   6    0    1  241    0    0]
 [  40   30    1    0  522    8]
 [  16    1    0    0    8 2171]]
